https://github.com/fedhere/PUI2017_fb55/blob/master/HW11_fb55/HW11_Assignment1_instructions.ipynb

Kernel: PUI2016_Python3

In [1]:
import pandas as pd
import numpy as np
import zipfile 
import os
import urllib
import geopandas as gpd
from scipy.cluster.vq import kmeans2, whiten
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy.spatial.distance import cdist, pdist
import matplotlib.pyplot as plt 

%matplotlib inline

In [4]:
#these commands can be run on the shell and get the data with the command wget
#the cell needs to be run only once
#unhash to run 

!for ((y=94; y<=99; y+=1)); do wget \
http://www2.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp$y\totals.zip; done

!for ((y=0; y<=1; y+=1)); do wget \
http://www2.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp0$y\totals.zip; done

!for ((y=2; y<=9; y+=1)); do wget \
http://www2.census.gov/econ200$y\/CBP_CSV/zbp0$y\totals.zip; done

!for ((y=10; y<=14; y+=1)); do wget \
http://www2.census.gov/econ20$y\/CBP_CSV/zbp$y\totals.zip; done

--2017-12-19 12:07:45--  http://www2.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp94totals.zip
Resolving www2.census.gov... 104.106.248.146, 2600:141b:13:295::208c, 2600:141b:13:288::208c
Connecting to www2.census.gov|104.106.248.146|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://www2.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp94totals.zip?sec_ak_reference=18.6683d717.1513703265.3044be78 [following]
--2017-12-19 12:07:45--  https://www2.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp94totals.zip?sec_ak_reference=18.6683d717.1513703265.3044be78
Connecting to www2.census.gov|104.106.248.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611524 (597K) [application/zip]
Saving to: “zbp94totals.zip”

100%[======================================>] 611,524     2.98M/s   in 0.2s    

2017-12-19 12:07:46 (2.98 MB/s) - “zbp94totals.zip” saved [611524/611524]

--2017-12-19 12:07:46--  http://www2.census.gov/Econ2001_And_Earlie

100%[======================================>] 620,754     3.09M/s   in 0.2s    

2017-12-19 12:07:49 (3.09 MB/s) - “zbp02totals.zip” saved [620754/620754]

--2017-12-19 12:07:49--  http://www2.census.gov/econ2003/CBP_CSV/zbp03totals.zip
Resolving www2.census.gov... 104.106.248.146, 2600:141b:13:295::208c, 2600:141b:13:288::208c
Connecting to www2.census.gov|104.106.248.146|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://www2.census.gov/econ2003/CBP_CSV/zbp03totals.zip?sec_ak_reference=18.4d83d717.1513703269.2605801 [following]
--2017-12-19 12:07:49--  https://www2.census.gov/econ2003/CBP_CSV/zbp03totals.zip?sec_ak_reference=18.4d83d717.1513703269.2605801
Connecting to www2.census.gov|104.106.248.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622790 (608K) [application/zip]
Saving to: “zbp03totals.zip”

100%[======================================>] 622,790     3.60M/s   in 0.2s    

2017-12-19 12:07:50 (3.6

HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://www2.census.gov/econ2012/CBP_CSV/zbp12totals.zip?sec_ak_reference=18.6683d717.1513703275.3045084d [following]
--2017-12-19 12:07:55--  https://www2.census.gov/econ2012/CBP_CSV/zbp12totals.zip?sec_ak_reference=18.6683d717.1513703275.3045084d
Connecting to www2.census.gov|104.106.248.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733541 (716K) [application/zip]
Saving to: “zbp12totals.zip”

100%[======================================>] 733,541     2.68M/s   in 0.3s    

2017-12-19 12:07:56 (2.68 MB/s) - “zbp12totals.zip” saved [733541/733541]

--2017-12-19 12:07:56--  http://www2.census.gov/econ2013/CBP_CSV/zbp13totals.zip
Resolving www2.census.gov... 104.106.248.146, 2600:141b:13:295::208c, 2600:141b:13:288::208c
Connecting to www2.census.gov|104.106.248.146|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://www2.census.gov/econ201

#### Make dataframe for all data

In [5]:
list1 = []

for i in range(94,100): 
    fname = 'zbp' + str(i) + 'totals.zip'
    zf = zipfile.ZipFile(fname)
    df1 = pd.read_csv(zf.open(fname.replace('.zip','.txt')))
    df1['year'] = i
    df1.columns = [x.lower() for x in df1.columns]
    df1 = df1[(df1['zip'].astype(str).str.startswith('1'))]
    df1 = df1[['zip', 'name', 'est', 'year']]
    list1.append(df1)
    
for i in range(00,15): 
    fname = 'zbp' + str(i).zfill(2) + 'totals.zip'
    zf = zipfile.ZipFile(fname)
    df1 = pd.read_csv(zf.open(fname.replace('.zip','.txt')))
    df1['year'] = i
    df1.columns = [x.lower() for x in df1.columns]
    df1 = df1[(df1['zip'].astype(str).str.startswith('1'))]
    df1 = df1[['zip', 'name', 'est', 'year']]
    list1.append(df1)
    
df1 = pd.concat(list1)

In [6]:
df1.head()

,zip,name,est,year
0,1001,"Agawam, MA",439,94
1,1002,"Amherst, MA",450,94
2,1003,"Amherst, MA",10,94
3,1004,"Amherst, MA",47,94
4,1005,"Barre, MA",92,94


In [7]:
df1.shape

(97171, 4)

In [8]:
df1.year = df1.year.astype(int)

In [9]:
df1.dtypes

zip      int64
name    object
est      int64
year     int64
dtype: object

In [10]:
df1.year = df1.year.replace({94:1994, 95:1995, 96:1996, 97:1997, 98:1998, 99:1999, 0:2000, 
                           1:2001, 2:2002, 3:2003, 4:2004, 5:2005, 6:2006, 
                           7:2007, 8:2008, 9:2009, 10:2010, 11:2011, 12:2012, 
                           13:2013, 14:2014})

In [11]:
df1[::4000]

,zip,name,est,year
0,1001,"Agawam, MA",439,1994
6604,18336,"Matamoras, PA",104,1994
5994,16725,"Custer City, PA",9,1995
5283,15227,"Pittsburgh, PA",397,1996
4619,13691,"THERESA, NY",34,1997
3859,12439,"HENSONVILLE, NY",16,1998
3242,10602,"WHITE PLAINS, NY",55,1999
7242,19930,"BETHANY BEACH, DE",182,1999
6604,18224,"FREELAND, PA",88,2000
5946,16686,"TYRONE, PA",246,2001


In [12]:
df1.shape

(97171, 4)

In [13]:
df_pivot = df1.pivot(index='zip', columns='year', values='est').reset_index()
df_pivot[::300]

year,zip,1994,1995,1996,1997,1998,1999,2000,2001,2002,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,1000,NaN,4.0,11.0,8.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,1803,1279.0,1295.0,1306.0,1355.0,1360.0,1357.0,1371.0,1351.0,1347.0,...,1362.0,1377.0,1442.0,1468.0,1437.0,1414.0,1432.0,1441.0,1470.0,1456.0
600,10518,66.0,66.0,66.0,67.0,69.0,68.0,66.0,67.0,60.0,...,78.0,92.0,72.0,70.0,66.0,67.0,67.0,68.0,69.0,66.0
900,11426,326.0,219.0,209.0,198.0,203.0,217.0,213.0,217.0,207.0,...,226.0,226.0,228.0,226.0,237.0,248.0,256.0,266.0,258.0,260.0
1200,12110,996.0,1005.0,1015.0,1036.0,1029.0,1049.0,1068.0,1054.0,1057.0,...,1132.0,1146.0,1163.0,1157.0,1130.0,1158.0,1140.0,1135.0,1157.0,1168.0
1500,12725,5.0,5.0,3.0,2.0,2.0,2.0,3.0,2.0,4.0,...,4.0,4.0,7.0,6.0,5.0,4.0,5.0,7.0,7.0,6.0
1800,13150,2.0,1.0,1.0,4.0,NaN,2.0,2.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2100,13776,20.0,21.0,18.0,16.0,16.0,15.0,14.0,14.0,13.0,...,12.0,15.0,14.0,14.0,11.0,10.0,12.0,14.0,12.0,9.0
2400,14527,296.0,314.0,319.0,323.0,319.0,330.0,314.0,329.0,326.0,...,344.0,336.0,345.0,336.0,326.0,330.0,340.0,344.0,342.0,341.0
2700,15060,12.0,12.0,12.0,14.0,14.0,11.0,12.0,15.0,16.0,...,15.0,13.0,12.0,13.0,14.0,14.0,13.0,10.0,10.0,12.0


In [14]:
df_pivot.shape

(4967, 22)

In [15]:
df_pivot = df_pivot.dropna(how='any')
df_pivot[::300]

year,zip,1994,1995,1996,1997,1998,1999,2000,2001,2002,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
1,1001,439.0,439.0,433.0,457.0,444.0,469.0,493.0,490.0,490.0,...,509.0,509.0,503.0,477.0,458.0,448.0,436.0,453.0,439.0,458.0
338,1876,612.0,642.0,653.0,657.0,661.0,678.0,706.0,726.0,746.0,...,778.0,778.0,787.0,765.0,724.0,738.0,705.0,710.0,707.0,706.0
705,10928,79.0,81.0,79.0,81.0,83.0,83.0,87.0,81.0,78.0,...,79.0,73.0,75.0,69.0,74.0,75.0,71.0,68.0,78.0,73.0
1080,11901,675.0,712.0,732.0,780.0,800.0,806.0,863.0,877.0,963.0,...,1038.0,1048.0,1080.0,1066.0,1070.0,1059.0,1047.0,1072.0,1078.0,1112.0
1415,12503,14.0,17.0,16.0,16.0,14.0,15.0,15.0,14.0,21.0,...,23.0,19.0,21.0,20.0,19.0,16.0,17.0,17.0,17.0,17.0
1739,13071,19.0,17.0,19.0,20.0,17.0,17.0,20.0,21.0,18.0,...,23.0,24.0,24.0,24.0,25.0,28.0,28.0,23.0,25.0,24.0
2070,13732,94.0,92.0,90.0,104.0,107.0,113.0,123.0,120.0,116.0,...,116.0,114.0,108.0,111.0,106.0,108.0,106.0,105.0,106.0,102.0
2399,14526,323.0,326.0,325.0,325.0,333.0,337.0,344.0,358.0,350.0,...,364.0,377.0,383.0,386.0,390.0,403.0,400.0,419.0,425.0,441.0
2733,15106,610.0,615.0,620.0,650.0,645.0,647.0,654.0,638.0,638.0,...,615.0,606.0,604.0,582.0,577.0,578.0,575.0,553.0,575.0,575.0
3085,15712,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,...,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [16]:
df_pivot.shape

(4341, 22)

#### Get zipcode shapefile

In [17]:
PUIdata = os.getenv('PUIDATA')

filename = "/ZIP_CODE_040114.shp"
url = "https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip"

if not os.path.isfile(PUIdata + filename):
    print("Downloading file now & unzipping to PUIdata")
    urllib.request.urlretrieve(url, PUIdata+filename)
    os.system("unzip -d %s nyc.zip"%(PUIdata))
    
else: 
    print("File already downloaded & unzipped in PUIdata")
    
    
shapefile = PUIdata + "/ZIP_CODE_040114.shp"

nycshp = gpd.GeoDataFrame.from_file(shapefile)
nycshp.head()

FionaValueError: No dataset found at path '/home/cusp/vmr286/PUIdata/ZIP_CODE_040114.shp' using drivers: *

In [ ]:
nycshp['ZIPCODE'] = nycshp['ZIPCODE'].astype(int)

#### Merge shapefile and df

In [ ]:
df = nycshp.merge(df_pivot, left_on="ZIPCODE", right_on="zip")
df.set_index('ZIPCODE')
print("DF Shape:", df.shape)
df[::20]

In [ ]:
df.columns

In [ ]:
df_ts = df[['ZIPCODE', 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 
            2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
            2012, 2013, 2014]]
df_ts = df_ts.set_index('ZIPCODE')
print("Shape of DF:", df_ts.shape)
df_ts.head()

In [ ]:
df_ts = df_ts.sort_index(ascending=True)
df_ts.head()

Note from Federica: 

you may need to clean your data: for some NYC zip codes there may be no info sanity check: you should have 20 (N_timestamps) datapoints per time series and about 250 zipcodes (Nzipcodes)

IMPORTANT: we talked about the importance of "whitening" your data. Whitenings decorrelates the data: it makes the features independent so that the data covariance matrix is the identity matrix. Whitening your data in time series analysis is in most cases wrong: you are modifying your time behaviour. This is because of the strong correlation between features (two consecutive time stamps for the same observation, the same zip code here, are strongly correlated). **Here instead you want to standardize your time series: subtract the mean and divide each time series (separately) by its standard deviation. As a sanity check (if you use skitlearn Kmeans or skitlearns kmeans2): you want your data array to be shaped Nzipcodes x Ntimestamps**

mydata.shape should be (Nzipcodes, Ntimestamps)

mydata[i].std() shoould be 1 for all i in range(len(Nzipcodes))

mydata[i].mean() should be ~0 for all i in range(len(Nzipcodes))

In [ ]:
df_ts2 = df_ts.reset_index()
zips = df_ts2.ZIPCODE
df_white = pd.DataFrame(whiten(df_ts2), columns=df_ts2.columns)
df_white['zips'] = zips
df_white = df_white.drop(['ZIPCODE'], axis=1)
df_white = df_white.set_index('zips')
df_white.head()

In [ ]:
df_scaled = preprocessing.scale(df_ts)
df_scaled.std(axis=0)

In [ ]:
wc = whiten(df_ts)

In [ ]:
df_white.head()

In [ ]:
## Function written by Prof. Sobolevsky in ADS 

def elbow(data1,K):
#data is your input as numpy form
#K is a list of number of clusters you would like to show.
    # Run the KMeans model and save all the results for each number of clusters
    KM = [KMeans(n_clusters=k).fit(data1) for k in K]
    
    # Save the centroids for each model with a increasing k
    centroids = [k.cluster_centers_ for k in KM]

    # For each k, get the distance between the data with each center. 
    D_k = [cdist(data1, cent, 'euclidean') for cent in centroids]
    
    # But we only need the distance to the nearest centroid since we only calculate dist(x,ci) for its own cluster.
    globals()['dist'] = [np.min(D,axis=1) for D in D_k]
    
    # Calculate the Average SSE.
    avgWithinSS = [sum(d)/data1.shape[0] for d in dist]
    
    
    # elbow curve
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(K, avgWithinSS, 'b*-')
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('Average within-cluster sum of squares')
    plt.title('Elbow for KMeans clustering')
    plt.show()
    
    
    # Total with-in sum of square plot. Another way to show the result.
    wcss = [sum(d**2) for d in dist]
    tss = sum(pdist(data1)**2)/data1.shape[0]
    bss = tss-wcss
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(K, bss/tss*100, 'b*-')
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('Percentage of variance explained')
    plt.title('Elbow for KMeans clustering')
    plt.show()

In [ ]:
X = np.asarray(df_white.iloc[:,:-1])
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]


for n_clusters in range_n_clusters:

    clusterer = KMeans(n_clusters=n_clusters, random_state=324)
    cluster_labels = clusterer.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)
    sample_silhouette_values = silhouette_samples(X, cluster_labels)
    
elbow(X, range(1,15))

Fig 1. The Elbows suggest using around somewhere around 6 clusters. 

## ASh score 50. Code has errors